In [1]:
# Using OpenCV template matching to find each individual market line
import cv2
import numpy as np
import pytesseract as tess
from PIL import Image as im


In [2]:
market_img = cv2.imread('cache/buy-orders/green_wood.png')
item_img = cv2.imread('resources/order_green_wood.png')
found_imgs = []

In [3]:
# # Displaying images for testing
# cv2.imshow('Market', item_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [4]:
# There are 6 comparison methods to choose from:
# TM_CCOEFF, TM_CCOEFF_NORMED, TM_CCORR, TM_CCORR_NORMED, TM_SQDIFF, TM_SQDIFF_NORMED
# You can see the differences at a glance here:
# https://docs.opencv.org/master/d4/dc6/tutorial_py_template_matching.html
result = cv2.matchTemplate(market_img, item_img, cv2.TM_CCOEFF_NORMED)

In [5]:
# Strictness of matching
threshold = 0.75

# Size of box to draw around matches
w = item_img.shape[1]
h = item_img.shape[0]

# Drawing a box around all matching objects?
yloc, xloc = np.where(result >= threshold)

In [6]:
# Drawing box around each found location, also cropping found boxes and adding to found_imgs array
for (x, y) in zip(xloc, yloc):
    cv2.rectangle(market_img, (x, y), (x + w, y + h), (0,255,255), 2)
    found_imgs.append(market_img[y:y+h, x:x+w])

In [7]:
# cv2.imshow('Market', market_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [8]:
# cv2.imshow('Order', found_imgs[0])
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [9]:
text = tess.image_to_string(found_imgs[0])

In [10]:
text

" \n\n \n\n0.64\n\nCommon\n\n670\n\n1d\n\nMonarch's Bluffs\n\n \n\n \n\x0c"

In [11]:
text_df = tess.image_to_data(found_imgs[0], output_type=tess.Output.DATAFRAME)

text_df = text_df.dropna()

In [12]:
text_df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
4,5,1,1,1,1,1,0,0,1200,2,95.000000,
8,5,1,2,1,1,1,0,0,2,71,95.000000,
12,5,1,3,1,1,1,409,28,28,14,96.851639,0.64
16,5,1,4,1,1,1,806,30,63,11,93.203789,Common
20,5,1,5,1,1,1,898,30,25,11,95.460495,670
24,5,1,6,1,1,1,1030,30,15,11,93.358971,1d
28,5,1,7,1,1,1,1094,31,58,10,79.482895,Monarch's
29,5,1,7,1,1,2,1156,31,31,10,86.159561,Bluffs
33,5,1,8,1,1,1,0,70,1200,1,95.000000,
37,5,1,9,1,1,1,1200,0,1,71,95.000000,


In [13]:
# This whole block is temporary to find best methods

# Changing data type from array to PIL Image object
temp_image = im.fromarray(found_imgs[3])

# Getting the size of the current object
w, h = temp_image.size

# Upscaling the image at same aspect ratio to improve tesseract accuracy
w *= 2
h *= 2

# Creating tuple that must be passed into resize function
newsize = (w, h)

# Resizing image using Bicubic upscaling
# https://pillow.readthedocs.io/en/stable/handbook/concepts.html#PIL.Image.BICUBIC
temp_image = temp_image.resize(newsize, resample=3)

# Extracting text from upscaled image
text_df = tess.image_to_data(temp_image, output_type=tess.Output.DATAFRAME)

# Dropping any rows with a null value extracted
text_df = text_df.dropna()


In [15]:
text_df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
4,5,1,1,1,1,1,0,0,2402,8,95.000000,
8,5,1,2,1,1,1,31,28,11,14,27.737022,Ps
10,5,1,2,1,2,1,29,27,87,77,45.330376,on
11,5,1,2,1,2,2,157,60,79,22,96.451950,Green
12,5,1,2,1,2,3,246,60,81,22,96.035332,Wood
13,5,1,2,1,2,4,817,52,58,42,96.359718,0.65
14,5,1,2,1,2,5,1002,56,7,28,37.958282,]
15,5,1,2,1,2,6,1119,75,21,4,79.757439,-
16,5,1,2,1,2,7,1242,72,20,2,22.941101,-
17,5,1,2,1,2,8,1376,72,22,4,64.909180,=


In [16]:
# need to resize each 

for img in found_imgs:
#     text_found_df = tess.image_to_data(found_imgs[img], output_type=tess.Output.DATAFRAME)
    print(found_imgs[img])

TypeError: only integer scalar arrays can be converted to a scalar index